In [ ]:
#Подключает пакет readxl, чтобы можно было читать .xlsx через read_excel().
library(readxl)
#Подключает пакет dplyr для удобной работы с таблицами: group_by, summarise, mutate, %>%.
library(dplyr)


#Читает Excel-файл Task11.xlsx и кладёт его в датафрейм df.
#В df должны быть столбцы: id, signal, response.
df <- read_excel("Task11.xlsx")

#Начинаем “цепочку” преобразований: берём df и передаём дальше.
rates <- df %>%

#Группируем строки по испытуемому id, чтобы считать показатели отдельно для каждого.
  group_by(id) %>%
#Создаём новую таблицу (по одной строке на каждого id) и считаем агрегаты.
  summarise(
      n_sig   = sum(signal == 1), #Считаем, сколько проб, где был сигнал
      n_noise = sum(signal == 0), #Считаем, сколько проб без сигнала (шум)
      hits = sum(signal == 1 & response == 1), #Считаем попадания: сигнал был (signal==1) и ответ “да”
      fas  = sum(signal == 0 & response == 1), #Считаем ложные тревоги: сигнала не было (signal==0), но ответ “да”
      H = hits / n_sig, #Считаем долю попаданий
      F = fas / n_noise #То же самое для доли ложных тревог
        )
rates

id,n_sig,n_noise,hits,fas,H,F
<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
1,480,520,339,150,0.7062500,0.2884615
2,520,480,388,124,0.7461538,0.2583333


In [ ]:
# Определяем z-функцию: переводит вероятность p (0..1) в z-оценку стандартной нормали
# (это обратная функция распределения нормали: Φ^{-1}(p))
z <- function(p) qnorm(p)

# Добавляем в таблицу rates показатели SDT по данным:
# d' — чувствительность; c_emp — эмпирический критерий (для справки)
rates <- rates %>%                      # берём таблицу rates и добавляем новые столбцы
  mutate(
    dprime = z(H) - z(F),               # d' = z(H) - z(F)
    c_emp  = -0.5 * (z(H) + z(F))       # c = -0.5*(z(H)+z(F)) (эмпирический критерий)
  )
rates



id,n_sig,n_noise,hits,fas,H,F,dprime,c_emp
<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,480,520,339,150,0.7062500,0.2884615,1.100347,0.007711179
2,520,480,388,124,0.7461538,0.2583333,1.310927,-0.006971543


In [ ]:
# Задаём параметры задачи (условия эксперимента и выплаты)
pS   <- 0.25    # доля (вероятность) сигнал-проб P(S)
pN   <- 0.75    # доля (вероятность) шум-проб P(N)
V_H  <-  3      # выигрыш за hit
V_FA <- -2      # штраф за false alarm (отрицательная полезность)
V_M  <-  0      # полезность за miss (если не задано в задаче — считаем 0)
V_CR <-  0      # полезность за correct rejection (если не задано — считаем 0)

# Считаем оптимальное β* (критерий по отношению правдоподобий) для максимизации денег
# Формула: β* = ((V_CR - V_FA) / (V_H - V_M)) * (P(N) / P(S))
beta_opt <- ((V_CR - V_FA) / (V_H - V_M)) * (pN / pS)

# По d' и β* вычисляем оптимальный критерий c_opt, затем оптимальные H* и F*
# При симметричной SDT-модели: z(H*) = d'/2 - c, z(F*) = -d'/2 - c
rates <- rates %>%                      # снова берём rates и добавляем оптимальные величины
  mutate(
    c_opt  = log(beta_opt) / dprime,    # оптимальный критерий: c = ln(β)/d'
    zH_opt = dprime / 2 - c_opt,        # z(H*) по формуле z(H*) = d'/2 - c
    zF_opt = -dprime / 2 - c_opt,       # z(F*) по формуле z(F*) = -d'/2 - c
    H_opt  = pnorm(zH_opt),             # переводим z(H*) обратно в вероятность: H* = Φ(zH*)
    F_opt  = pnorm(zF_opt),             # переводим z(F*) обратно в вероятность: F* = Φ(zF*)

    # Ожидаемый выигрыш на одну пробу:
    # сигнал случается с вероятностью pS, тогда hit бывает с вероятностью H_opt и приносит V_H
    # шум случается с вероятностью pN, тогда FA бывает с вероятностью F_opt и приносит V_FA
    EV_trial = pS * H_opt * V_H + pN * F_opt * V_FA,

    # Ожидаемый выигрыш на 1000 проб (как в задании)
    EV_1000  = EV_trial * 1000
  )

# Печатаем итоговую таблицу со всеми рассчитанными показателями
rates

















id,n_sig,n_noise,hits,fas,H,F,dprime,c_emp,c_opt,zH_opt,zF_opt,H_opt,F_opt,EV_trial,EV_1000
<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,480,520,339,150,0.7062500,0.2884615,1.100347,0.007711179,0.6299350,-0.07976141,-1.180109,0.4682135,0.1189785,0.1726924,172.6924
2,520,480,388,124,0.7461538,0.2583333,1.310927,-0.006971543,0.5287456,0.12671808,-1.184209,0.5504182,0.1181651,0.2355660,235.5660
